In [16]:
import torch
import os

In [17]:

os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
os.environ["CUDA_DEVICE_ORDER"]= "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= '2'


In [18]:
from natsort import natsorted
from tqdm import tqdm
from glob import glob
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D

all_blocks = []
all_labels = []

all_files_path = natsorted(glob('merged_data_pkl/*'))

for path in tqdm(all_files_path):
    with open(path, 'rb') as file:
        data_dict = pickle.load(file)
        all_blocks.append(data_dict['data'])
        all_labels.append(data_dict['label'])

X = np.array(all_blocks)
Y = np.array(all_labels)

print(X.shape, Y.shape)


100%|██████████| 8400/8400 [00:00<00:00, 16812.35it/s]


(8400, 32, 250) (8400,)


In [19]:
# Assuming `X` is loaded and shaped [batch_size, 1, 32, 250]
X = X.reshape(X.shape[0], 250, 32)  # Reshape X to [batch_size, sequence_length, features]

# Convert to PyTorch tensor
tensor_X = torch.Tensor(X)
tensor_Y = torch.LongTensor(Y)  # Assuming Y is your label tensor


In [20]:
from torch.utils.data import TensorDataset, DataLoader

# Create dataset and dataloader
dataset = TensorDataset(tensor_X, tensor_Y)
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)


In [21]:
import torch
import torch.nn as nn
import torch.optim as optim

class SoundLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(SoundLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # Define an LSTM layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        
        # Define a fully connected layer for output
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        
        # Forward propagate the LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
# Parameters
input_dim = 32
hidden_dim = 128
output_dim = 10  # Assuming you have 10 different animal classes
num_layers = 2

model = SoundLSTM(input_dim, hidden_dim, output_dim, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [24]:
def calculate_accuracy(model, data_loader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    model.train()  # Set the model back to training mode
    return correct / total


In [26]:
from torch.utils.data import random_split, DataLoader

# Assuming 'dataset' is an instance of a dataset containing all your data
total_size = len(dataset)
train_size = int(0.8 * total_size)  # Using 80% for training
val_size = total_size - train_size  # Remaining 20% for validation

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders for each dataset
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [27]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # Calculate validation accuracy
        val_accuracy = calculate_accuracy(model, val_loader, device)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}, Val Accuracy: {val_accuracy * 100:.2f}%')

# Assuming the device is already defined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Train the model with validation accuracy reporting
train_model(model, train_loader, val_loader, criterion, optimizer, 20, device)


Epoch [1/20], Loss: 2.2103, Val Accuracy: 13.04%
Epoch [2/20], Loss: 2.1758, Val Accuracy: 13.63%
Epoch [3/20], Loss: 2.1481, Val Accuracy: 15.36%
Epoch [4/20], Loss: 2.1134, Val Accuracy: 15.12%
Epoch [5/20], Loss: 2.0561, Val Accuracy: 15.83%
Epoch [6/20], Loss: 1.9930, Val Accuracy: 15.30%
Epoch [7/20], Loss: 1.8999, Val Accuracy: 15.12%
Epoch [8/20], Loss: 1.7956, Val Accuracy: 15.12%
Epoch [9/20], Loss: 1.6743, Val Accuracy: 15.65%
Epoch [10/20], Loss: 1.5390, Val Accuracy: 16.07%
Epoch [11/20], Loss: 1.3876, Val Accuracy: 15.83%
Epoch [12/20], Loss: 1.2263, Val Accuracy: 15.60%
Epoch [13/20], Loss: 1.0776, Val Accuracy: 15.60%
Epoch [14/20], Loss: 0.9196, Val Accuracy: 16.07%
Epoch [15/20], Loss: 0.7701, Val Accuracy: 14.46%
Epoch [16/20], Loss: 0.6353, Val Accuracy: 15.89%
Epoch [17/20], Loss: 0.5076, Val Accuracy: 15.48%
Epoch [18/20], Loss: 0.4320, Val Accuracy: 14.52%
Epoch [19/20], Loss: 0.3495, Val Accuracy: 16.01%
Epoch [20/20], Loss: 0.2693, Val Accuracy: 14.94%


In [12]:
test_data = pd.read_excel('/home/project/new/Test.xlsx')

In [14]:
import numpy as np
import pandas as pd
import torch

# Assuming `test_data` is already loaded and is a Pandas DataFrame
all_test_blocks = []
block_height = 32
num_columns_in_test = 750  # Columns per test block

# Iterate over the dataset in chunks of 32 rows
for start_row in range(0, test_data.shape[0], block_height):
    if start_row + block_height > test_data.shape[0]:
        continue  # Skip incomplete blocks
    chunk = test_data.iloc[start_row:start_row + block_height]

    # Split each 750-column block into three 250-column blocks
    for i in range(3):  # Since 750/250 = 3
        start_col = i * 250
        end_col = start_col + 250
        block = chunk.iloc[:, start_col:end_col]
        all_test_blocks.append(block.values.reshape(1, block_height, 250))  # Adding channel dimension

# Convert blocks to PyTorch tensors
X_test_blocks = torch.tensor(all_test_blocks, dtype=torch.float)


/tmp/ipykernel_1261012/2964036573.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  X_test_blocks = torch.tensor(all_test_blocks, dtype=torch.float)


In [15]:
# Assuming X_test is loaded and initially shaped as [batch_size, 1, 32, 250]
X_test = X_test.reshape(X_test.shape[0], 250, 32)  # Reshape to [batch_size, sequence_length, features]

# Convert to PyTorch tensor
tensor_X_test = torch.Tensor(X_test)
tensor_Y_test = torch.LongTensor(Y_test)  # Assuming Y_test contains the labels

# Create dataset and dataloader for the test data
test_dataset = TensorDataset(tensor_X_test, tensor_Y_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


NameError: name 'X_test' is not defined

In [ ]:
# Assuming the model is already defined as `SoundCNN`
model = SoundCNN()
model.load_state_dict(torch.load('/home/project/new/SoundCNN_model.pth')['model_state_dict'])

# Transfer model to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set the model to evaluation mode


SoundCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=7936, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
  (relu): ReLU()
)

In [ ]:
X_test_blocks = X_test_blocks.to(device)  # Move the test data to the same device as the model
with torch.no_grad():  # Ensure no gradients are computed during inference
    predictions = model(X_test_blocks)
    predicted_classes = torch.argmax(predictions, dim=1).cpu().numpy()  # Move predictions back to CPU and convert to numpy


In [ ]:
import pandas as pd

# `predicted_classes` is the numpy array containing the class indices predicted by the model
predictions_df = pd.DataFrame(predicted_classes, columns=['PredictedClass'])


In [ ]:
# Specify the path and name of the CSV file
csv_file_path = './predictions.csv'

# Save the DataFrame to a CSV file without the index column
predictions_df.to_csv(csv_file_path, index=False)


In [ ]:
label_map = {
    0: "Tiger",
    1: "Snake",
    2: "Wolf",
    3: "Bear",
    4: "Rabbit",
    5: "Monkey",
    6: "Eagle",
    7: "Dolphin",
    8: "Koala"
}


In [ ]:
import pandas as pd

# Assuming `predicted_classes` is already available as a numpy array
predictions_df = pd.DataFrame(predicted_classes, columns=['PredictedClass'])

# Replace numeric labels with animal names using the mapping
predictions_df['PredictedClass'] = predictions_df['PredictedClass'].replace(label_map)


In [ ]:
# Specify the path and name of the CSV file
csv_file_path = './animal_predictions.csv'

# Save the DataFrame to a CSV file without the index column
predictions_df.to_csv(csv_file_path, index=False)


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import random

# Load predictions
predictions_df = pd.read_csv('./animal_predictions.csv')

# Function to determine the majority or random in case of a tie
def majority_or_random(labels):
    count = Counter(labels)
    max_freq = max(count.values())
    candidates = [label for label, freq in count.items() if freq == max_freq]
    return random.choice(candidates)

# Group predictions in groups of 3 and apply the function
grouped_labels = []
for i in range(0, len(predictions_df), 3):
    labels = predictions_df['PredictedClass'][i:i+3].tolist()
    if len(labels) == 3:  # Ensure it's a full group of 3
        grouped_label = majority_or_random(labels)
        grouped_labels.append(grouped_label)

# Create new DataFrame
final_predictions_df = pd.DataFrame(grouped_labels, columns=['MajorityVotedClass'])


In [ ]:
# Save the DataFrame to a CSV file
final_csv_file_path = './final_animal_predictions.csv'
final_predictions_df.to_csv(final_csv_file_path, index=False)


In [ ]:
import pandas as pd

# Load the CSV file
file_path = './final_animal_predictions.csv'
predictions_df = pd.read_csv(file_path)


In [ ]:
# Generate the 'ID' column
predictions_df['ID'] = ['id_' + str(index + 1) for index in predictions_df.index]


In [ ]:
# Reorder columns to make 'ID' the first column
column_order = ['ID', 'MajorityVotedClass']
predictions_df = predictions_df[column_order]


In [ ]:
# Save the updated DataFrame to a new CSV file
updated_csv_file_path = './updated_final_animal_predictions.csv'
predictions_df.to_csv(updated_csv_file_path, index=False)
